# Comparing methods to simulate radial Ornstein-Uhlenbeck bridge 
This is a specific case of an interest rates model proposed by Aït-Sahalia and Lo (Journal of Finance, 1998)

In [ ]:
import DiffusionBridge as db
import torch
import matplotlib.pyplot as plt
from scipy.special import iv, ivp
from torch.distributions.gamma import Gamma
plt.style.use('ggplot')

In [ ]:
# problem settings
d = 1
theta = torch.tensor(4.0)
f = lambda t,x: theta / x - x 
sigma = torch.tensor(1.0)
T = torch.tensor(1.0)
M = 50
diffusion = db.diffusion.model(f, sigma, d, T, M)

# transition density
log_transition_density = lambda t,x,x0: theta * torch.log(x / x0) + 0.5 * torch.log(x * x0) - x**2 \
                                    + (theta + 0.5) * t - torch.log(torch.sinh(t)) \
                                    - (x**2 + x0**2) / (torch.exp(2.0 * t) - 1.0) \
                                    + torch.log(iv(theta - 0.5, x * x0 / torch.sinh(t)))

score_transition = lambda t,x,x0: theta / x + 1.0 / (2.0 * x) - 2.0 * x - 2.0 * x / (torch.exp(2.0 * t) - 1.0) \
                                 + (1.0 / iv(theta - 0.5, x * x0 / torch.sinh(t))) * ivp(theta - 0.5, x * x0 / torch.sinh(t)) * x0 / torch.sinh(t)

# h-transform
grad_logh = lambda t,x,xT: -theta / x + 1.0 / (2.0 * x) - 2.0 * x / (torch.exp(2.0 * (T-t)) - 1.0) \
                        + (1.0 / iv(theta - 0.5, xT * x / torch.sinh(T-t))) * ivp(theta - 0.5, xT * x / torch.sinh(T-t)) * xT / torch.sinh(T-t)

# score marginal 
score_marginal = lambda t,x,x0,xT: score_transition(t, x, x0) + grad_logh(t, x, xT)

# repetitions
N = 2**10
R = 100


In [ ]:
# initial and terminal conditions
initial_condition = [1.5, 1.5, 3.0]
terminal_condition = [1.0, 2.5, 5.0]

In [ ]:
# learn backward diffusion bridge process with score matching
distribution_X0 = Gamma(torch.tensor(5.0), torch.tensor(2.0))
simulate_X0 = lambda n: distribution_X0.sample((n, )).reshape(n, d)
XT = []
epsilon = 1.0
minibatch = 100
num_initial_per_batch = 10
num_iterations = 1000
learning_rate = 0.01
ema_momentum = 0.99

output = diffusion.learn_full_score_transition(simulate_X0, XT, epsilon, minibatch, num_initial_per_batch, num_iterations, learning_rate, ema_momentum)
score_transition_net = output['net']
loss_values_transition = output['loss']

In [ ]:
# simulate modified backward diffusion bridge (MBDB) process with approximate score
MBDB = {'ess' : torch.zeros(3,R), 'logestimate' : torch.zeros(3,R), 'acceptrate' : torch.zeros(3,R)}

for c in range(3):
    X0 = initial_condition[c] * torch.ones(d)
    XT = terminal_condition[c] * torch.ones(d)

    for r in range(R):
        with torch.no_grad():
            output = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, N, modify = True, full_score = True)
            trajectories = output['trajectories']
            log_proposal = output['logdensity']
        log_target = diffusion.law_bridge(trajectories)
        log_weights = log_target - log_proposal

        # importance sampling 
        max_log_weights = torch.max(log_weights)
        weights = torch.exp(log_weights - max_log_weights)
        norm_weights = weights / torch.sum(weights)
        ess = 1.0 / torch.sum(norm_weights**2)
        log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
        MBDB['ess'][c,r] = ess
        MBDB['logestimate'][c,r] = log_transition_estimate

        # independent Metropolis-Hastings
        initial = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, 1, modify = True, full_score = True)
        current_trajectory = initial['trajectories']
        current_log_proposal = initial['logdensity'] 
        current_log_target = diffusion.law_bridge(current_trajectory)
        current_log_weight = current_log_target - current_log_proposal
        num_accept = 0
        for n in range(N):
            proposed_trajectory = trajectories[n, :, :]
            proposed_log_weight = log_weights[n]
            log_accept_prob = proposed_log_weight - current_log_weight

            if (torch.log(torch.rand(1)) < log_accept_prob):
                current_trajectory = proposed_trajectory.clone()
                current_log_weight = proposed_log_weight.clone()  
                num_accept += 1
        accept_rate = num_accept / N
        MBDB['acceptrate'][c,r] = accept_rate
        
        # print
        print('Initial: ' + str(initial_condition[c]) + 
            ' Terminal: ' + str(terminal_condition[c]) + 
            ' Repeat: ' + str(r) + 
            ' ESS%: ' + str(float(ess * 100 / N)) + 
            ' log-transition density: ' + str(float(log_transition_estimate)), 
            ' Accept rate: ' + str(float(accept_rate)))


In [ ]:
# forward diffusion (FD) method of Pedersen (1995)
drift = f
FD = {'ess' : torch.zeros(3,R), 'logestimate' : torch.zeros(3,R), 'acceptrate' : torch.zeros(3,R)}

for c in range(3):
    X0 = initial_condition[c] * torch.ones(d)
    XT = terminal_condition[c] * torch.ones(d)
    
    for r in range(R):
        output = diffusion.simulate_proposal_bridge(drift, X0, XT, N)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
        log_target = diffusion.law_bridge(trajectories) 
        log_weights = log_target - log_proposal

        # importance sampling
        max_log_weights = torch.max(log_weights)
        weights = torch.exp(log_weights - max_log_weights)
        norm_weights = weights / torch.sum(weights)
        ess = 1.0 / torch.sum(norm_weights**2)    
        log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
        FD['ess'][c,r] = ess
        FD['logestimate'][c,r] = log_transition_estimate

        # independent Metropolis-Hastings
        initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1)
        current_trajectory = initial['trajectories']
        current_log_proposal = initial['logdensity'] 
        current_log_target = diffusion.law_bridge(current_trajectory)
        current_log_weight = current_log_target - current_log_proposal
        num_accept = 0
        for n in range(N):
            proposed_trajectory = trajectories[n, :, :]
            proposed_log_weight = log_weights[n]
            log_accept_prob = proposed_log_weight - current_log_weight

            if (torch.log(torch.rand(1)) < log_accept_prob):
                current_trajectory = proposed_trajectory.clone()
                current_log_weight = proposed_log_weight.clone()  
                num_accept += 1
        accept_rate = num_accept / N
        FD['acceptrate'][c,r] = accept_rate
        
        # print
        print('Initial: ' + str(initial_condition[c]) + 
            ' Terminal: ' + str(terminal_condition[c]) + 
            ' Repeat: ' + str(r) + 
            ' ESS%: ' + str(float(ess * 100 / N)) + 
            ' log-transition density: ' + str(float(log_transition_estimate)), 
            ' Accept rate: ' + str(float(accept_rate)))


In [ ]:
# modified diffusion bridge (MDB) method of Durham and Gallant (2002)
drift = lambda t,x: (XT - x) / (T - t)
MDB = {'ess' : torch.zeros(3,R), 'logestimate' : torch.zeros(3,R), 'acceptrate' : torch.zeros(3,R)}

for c in range(3):
    X0 = initial_condition[c] * torch.ones(d)
    XT = terminal_condition[c] * torch.ones(d)

    for r in range(R):
        output = diffusion.simulate_proposal_bridge(drift, X0, XT, N, modify = True)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
        log_target = diffusion.law_bridge(trajectories) 
        log_weights = log_target - log_proposal
        
        # importance sampling
        max_log_weights = torch.max(log_weights)
        weights = torch.exp(log_weights - max_log_weights)
        norm_weights = weights / torch.sum(weights)
        ess = 1.0 / torch.sum(norm_weights**2)
        log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
        MDB['ess'][c,r] = ess
        MDB['logestimate'][c,r] = log_transition_estimate

        # independent Metropolis-Hastings
        initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1, modify = True)
        current_trajectory = initial['trajectories']
        current_log_proposal = initial['logdensity'] 
        current_log_target = diffusion.law_bridge(current_trajectory)
        current_log_weight = current_log_target - current_log_proposal
        num_accept = 0
        for n in range(N):
            proposed_trajectory = trajectories[n, :, :]
            proposed_log_weight = log_weights[n]
            log_accept_prob = proposed_log_weight - current_log_weight

            if (torch.log(torch.rand(1)) < log_accept_prob):
                current_trajectory = proposed_trajectory.clone()
                current_log_weight = proposed_log_weight.clone()  
                num_accept += 1
        accept_rate = num_accept / N
        MDB['acceptrate'][c,r] = accept_rate

        # print
        print('Initial: ' + str(initial_condition[c]) + 
            ' Terminal: ' + str(terminal_condition[c]) + 
            ' Repeat: ' + str(r) + 
            ' ESS%: ' + str(float(ess * 100 / N)) + 
            ' log-transition density: ' + str(float(log_transition_estimate)), 
            ' Accept rate: ' + str(float(accept_rate)))

In [ ]:
# diffusion bridge proposal of Clark (1990) and Delyon and Hu (2006) (CDH)
drift = lambda t,x: f(t,x) + (XT - x) / (T - t)
CDH = {'ess' : torch.zeros(3,R), 'logestimate' : torch.zeros(3,R), 'acceptrate' : torch.zeros(3,R)}

for c in range(3):
    X0 = initial_condition[c] * torch.ones(d)
    XT = terminal_condition[c] * torch.ones(d)
    
    for r in range(R):
        output = diffusion.simulate_proposal_bridge(drift, X0, XT, N, modify = False)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
        log_target = diffusion.law_bridge(trajectories) 
        log_weights = log_target - log_proposal

        # importance sampling
        max_log_weights = torch.max(log_weights)
        weights = torch.exp(log_weights - max_log_weights)
        norm_weights = weights / torch.sum(weights)
        ess = 1.0 / torch.sum(norm_weights**2)
        log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
        CDH['ess'][c,r] = ess
        CDH['logestimate'][c,r] = log_transition_estimate

        # independent Metropolis-Hastings
        initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1, modify = False)
        current_trajectory = initial['trajectories']
        current_log_proposal = initial['logdensity'] 
        current_log_target = diffusion.law_bridge(current_trajectory)
        current_log_weight = current_log_target - current_log_proposal
        num_accept = 0
        for n in range(N):
            proposed_trajectory = trajectories[n, :, :]
            proposed_log_weight = log_weights[n]
            log_accept_prob = proposed_log_weight - current_log_weight

            if (torch.log(torch.rand(1)) < log_accept_prob):
                current_trajectory = proposed_trajectory.clone()
                current_log_weight = proposed_log_weight.clone()  
                num_accept += 1
        accept_rate = num_accept / N
        CDH['acceptrate'][c,r] = accept_rate

        # print
        print('Initial: ' + str(initial_condition[c]) + 
            ' Terminal: ' + str(terminal_condition[c]) + 
            ' Repeat: ' + str(r) + 
            ' ESS%: ' + str(float(ess * 100 / N)) + 
            ' log-transition density: ' + str(float(log_transition_estimate)), 
            ' Accept rate: ' + str(float(accept_rate)))

In [ ]:
# compare ESS
for c in range(3):
    print('Initial: ' + str(initial_condition[c]) + ' Terminal: ' + str(terminal_condition[c])) 
    print('FD ESS%: ' + str(float(torch.mean(FD['ess'][c,:]) * 100 / N)))
    print('MDB ESS%: ' + str(float(torch.mean(MDB['ess'][c,:]) * 100 / N)))
    print('CDH ESS%: ' + str(float(torch.mean(CDH['ess'][c,:]) * 100 / N)))
    print('MBDB ESS%: ' + str(float(torch.mean(MBDB['ess'][c,:]) * 100 / N)))


In [ ]:
# compare RMSE of log-transition density 
for c in range(3):
    X0 = initial_condition[c] * torch.ones(d)
    XT = terminal_condition[c] * torch.ones(d)
    print('Initial: ' + str(initial_condition[c]) + ' Terminal: ' + str(terminal_condition[c])) 
    print('FD RMSE: ' + str(float(torch.sqrt(torch.mean((FD['logestimate'][c,:] - log_transition_density(T, XT, X0))**2)))))
    print('MDB RMSE: ' + str(float(torch.sqrt(torch.mean((MDB['logestimate'][c,:] - log_transition_density(T, XT, X0))**2)))))
    print('CDH RMSE: ' + str(float(torch.sqrt(torch.mean((CDH['logestimate'][c,:] - log_transition_density(T, XT, X0))**2)))))
    print('MBDB RMSE: ' + str(float(torch.sqrt(torch.mean((MBDB['logestimate'][c,:] - log_transition_density(T, XT, X0))**2)))))

In [ ]:
# compare indepedent Meteropolis-Hastings acceptance rate
for c in range(3):
    print('Initial: ' + str(initial_condition[c]) + ' Terminal: ' + str(terminal_condition[c])) 
    print('FD acceptance%: ' + str(float(torch.mean(FD['acceptrate'][c,:] * 100))))
    print('MDB acceptance%: ' + str(float(torch.mean(MDB['acceptrate'][c,:] * 100))))
    print('CDH acceptance%: ' + str(float(torch.mean(CDH['acceptrate'][c,:] * 100))))
    print('MBDB acceptance%: ' + str(float(torch.mean(MBDB['acceptrate'][c,:] * 100))))


In [ ]:
# store results
results = {'FD': FD, 
           'MDB': MDB, 
           'CDH': CDH, 
           'MBDB': MBDB}

torch.save(results, 'radial_T1.pt')